# Basic Stats 2 (Assignment - 4)

Problem Statement : Hospital Patient Data Analysis

 Context:
- A hospital maintains patient records including admission details, department, diagnosis, doctor, and bill amount. You have two datasets: one with patient info and another with billing details. Some patients have blank bill amounts, and there are multiple rows for the same patient due to follow-ups.


Expected Outcome:
- Final cleaned dataset with accurate billing info.
- All missing values handled, merged dataset across PatientID.
- Ability to perform further analytics on department-wise revenue or doctor performance.

### Solutions

### 1.	Load the patient dataset and show summary with info().

In [1]:
import pandas as pd 
df = pd.read_csv("Patient_Data.csv") #Load Dataset

In [17]:
df.head(10)

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45
5,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00


In [3]:
df.info() #info of dataset

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


### 2.	Select only the columns relevant for billing: ['PatientID', 'Department', 'Doctor', 'BillAmount'].

In [4]:
bill_df =  df[['PatientID', 'Department', 'Doctor', 'BillAmount']] #select billing relevant columns

In [5]:
bill_df

,PatientID,Department,Doctor,BillAmount
0,101,Cardiology,Dr. Smith,5000.0
1,102,Neurology,Dr. John,NaN
2,103,Orthopedics,Dr. Lee,7500.0
3,104,Cardiology,Dr. Smith,6200.0
4,105,Dermatology,Dr. Rose,NaN
5,101,Cardiology,Dr. Smith,5000.0


### 3.	Drop administrative columns like ['ReceptionistID', 'CheckInTime'].

In [6]:
df2 = df.drop(['ReceptionistID', 'CheckInTime'], axis = 1) #Drop Columns 

In [7]:
df2

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN
5,101,Alice,Cardiology,Dr. Smith,5000.0


### 4.	Use groupby to find total bill amount per department.

In [8]:
total_bill_per_dept = df.groupby('Department', as_index=False)['BillAmount'].sum() #Gropby totalbill per departmnet
total_bill_per_dept

,Department,BillAmount
0,Cardiology,16200.0
1,Dermatology,0.0
2,Neurology,0.0
3,Orthopedics,7500.0


### 5.	Remove duplicate patient records based on PatientID.

In [9]:
df3 = df2.drop_duplicates(subset='PatientID') #Remove Duplicates Based On PatientID
df3

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


### 6.	Fill missing BillAmount values with the mean bill amount.

In [10]:
df3["BillAmount"] = df3["BillAmount"].fillna(df["BillAmount"].mean()) #Filling Null Values with mean 
df3

C:\Users\shivi\AppData\Local\Temp\ipykernel_7336\3059806728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3["BillAmount"] = df3["BillAmount"].fillna(df["BillAmount"].mean()) #Filling Null Values with mean


,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,5925.0
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,5925.0


### 7.	Merge the billing dataset with patient dataset on PatientID.

In [11]:
billing_df = pd.read_csv("Billing_Data.csv") #Load Another Dataset
billing_df

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [12]:
merged_df = pd.merge(billing_df, df3, on='PatientID') #Merge Both Datasets
merged_df

,PatientID,InsuranceCovered,FinalAmount,Name,Department,Doctor,BillAmount
0,101,2000,3000,Alice,Cardiology,Dr. Smith,5000.0
1,102,1500,3500,Bob,Neurology,Dr. John,5925.0
2,103,2500,5000,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,3000,3200,David,Cardiology,Dr. Smith,6200.0
4,105,1000,4000,Eva,Dermatology,Dr. Rose,5925.0


### 8.	Concatenate an additional DataFrame that contains new patients for the current week (row-wise).

In [13]:
new_patients = pd.DataFrame({
    'PatientID': [501, 502],
    'Name': ['Rio', 'Eros'], 
    'Department': ['Cardiology', 'Orthopedics'],
    'Doctor': ['Dr. Shah', 'Dr. Mehta'],
    'BillAmount': [12000, 15000]
})  #Make New DataFrame

In [14]:
final_patient_df = pd.concat([merged_df, new_patients], axis=0, ignore_index=True) #Concatinate  DataFrame
final_patient_df

,PatientID,InsuranceCovered,FinalAmount,Name,Department,Doctor,BillAmount
0,101,2000.0,3000.0,Alice,Cardiology,Dr. Smith,5000.0
1,102,1500.0,3500.0,Bob,Neurology,Dr. John,5925.0
2,103,2500.0,5000.0,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,3000.0,3200.0,David,Cardiology,Dr. Smith,6200.0
4,105,1000.0,4000.0,Eva,Dermatology,Dr. Rose,5925.0
5,501,NaN,NaN,Rio,Cardiology,Dr. Shah,12000.0
6,502,NaN,NaN,Eros,Orthopedics,Dr. Mehta,15000.0


### 9.	Concatenate new billing category columns like ['InsuranceCovered', 'FinalAmount'] (column-wise).

In [15]:
billing_columns = pd.DataFrame({
    'InsuranceCovered': ['Yes'] * len(final_patient_df),
    'FinalAmount': final_patient_df['BillAmount'] * 0.8
})

final_df = pd.concat(
    [final_patient_df, billing_columns],
    axis=1
)

In [16]:
final_df

,PatientID,InsuranceCovered,FinalAmount,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,2000.0,3000.0,Alice,Cardiology,Dr. Smith,5000.0,Yes,4000.0
1,102,1500.0,3500.0,Bob,Neurology,Dr. John,5925.0,Yes,4740.0
2,103,2500.0,5000.0,Charlie,Orthopedics,Dr. Lee,7500.0,Yes,6000.0
3,104,3000.0,3200.0,David,Cardiology,Dr. Smith,6200.0,Yes,4960.0
4,105,1000.0,4000.0,Eva,Dermatology,Dr. Rose,5925.0,Yes,4740.0
5,501,NaN,NaN,Rio,Cardiology,Dr. Shah,12000.0,Yes,9600.0
6,502,NaN,NaN,Eros,Orthopedics,Dr. Mehta,15000.0,Yes,12000.0
